In [ ]:

#newIndex=[[keyword ,[url1,url2,...,],# of links], [keyword2 ,[url2,url3,...,],# of links]]

def union(list1, list2):
  for element in list2:
    if element not in list1:
      list1.append(element)
  return list1

def addToIndex(index,keyword,url): #This procedure is adjusted to the new type of index.
  presentKeyword = False #To search for each keyword in the index structure, this boolean is assigned.
  for cell in index:
    if cell[0] == keyword:
      union(cell[1],[url])
      cell[2] = len(cell[1])
      presentKeyword=True
  if not presentKeyword:
    index.append([keyword, [url], 1])
  return index

def getPage(url):
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""

def addPageToIndex(index, url,content):
  contentList=content.split()
  for element in contentList:
    addToIndex(index,element,url)
  return index

def getNextTarget(page):
  startLink=page.find('<a href=')
  if startLink==-1:
    return None,0
  startQuote = page.find('"', startLink+1)
  endQuote= page.find('"', startQuote+1 )
  url= page[startQuote+1: endQuote]
  return url, endQuote

def getAllLinks(page):
  urlList=[]
  while True:
    myUrl, lastQuote = getNextTarget(page)
    if myUrl:
      page=page[lastQuote:]
      urlList.append(myUrl)
    else:
      break
  return urlList

def crawlWeb(seed):
  tocrawl=[seed]
  crawled=[]
  index=[]
  graph={}
  while tocrawl:
    page=tocrawl.pop()
    if page not in crawled:
      content=getPage(page)
      addPageToIndex(index,page,content)
      graph[page]=getAllLinks(page)
      union(tocrawl,getAllLinks(getPage(page)))
      crawled.append(page)
  return index,graph

index, graph = crawlWeb("https://searchengineplaces.com.tr/") #Assignment of index and graph to the output of crawlWeb

def computeRanks (graph) :
  d = 0.8 # damping factor
  N = len( graph ) # number of pages
  numloops = 10 # effects the accuracy
  ranks = {}
  for page in graph :
    ranks [ page ] = 1/ N

  for i in range (0 , numloops ) :
    newranks = {}
    for page in graph :
      newrank = (1-d ) / N
      for node in graph :
        if page in graph [ node ]:
          newrank = newrank + d*(ranks[node]/len(graph(node)))
      newranks [ page ] = newrank
    ranks = newranks
  return newranks

ranks = computeRanks(graph) #Assignment of the ranks to the output of computeRanks

def lookUpPageRank(index,keyword,ranks): #With this fuction, the related urls of the given keyword is found.
  readytoSort={}
  numberOfLinks = 0
  keywords=[cell[0] for cell in index] #All keywords are assigned to a list
  if keyword not in keywords:
    return "No result found."
  if keyword in keywords:
     for cell in index:
      if keyword==cell[0]:
        unsortedList = cell[1]
        numberOfLinks = len(cell[1])
        for links in unsortedList:
         readytoSort[links]=ranks[links]

  sortedList = sorted(readytoSort, key=readytoSort.get, reverse=True)
  print(f'Keyword: {keyword}')
  print(f'{numberOfLinks} result(s) found.')
  print('Urls: ')
  for url in sortedList:
      print(url)

def lookupLink(index, url): #With this function, the related keywords of the given url is found.
  keywordsOfLinks = []
  allLinks=[]
  for cell in index:
    allLinks= union(allLinks,cell[1])

  if url in allLinks:
    for cell in index:
      if url in cell[1]:
        keywordsOfLinks = union(keywordsOfLinks,[cell[0]])
    print(f"Link:  {url}")
    print(f'{len(keywordsOfLinks)} keyword(s) found.')

    for element in keywordsOfLinks:
      print(element)
  else:
    return "No result found."

lookUpPageRank(index,"<!DOCTYPE",ranks)

lookupLink(index, "http://www.searchengineplaces.com.tr/oktayrecommends.html")